In [1]:
import pandas as pd

In [58]:
data = pd.read_stata('../02. Datasets/ttest_with_result_mean_std.dta')
data.columns = data.iloc[0]
# Drop the first row
data = data.drop(0).reset_index(drop=True)

In [59]:
for i in range(1, 12):
    data.loc[i, 'SD'] = data.loc[i, 'Mean']
    data.loc[i, 'Mean'] = data.loc[i, 'Variable']
    data.loc[i, 'Variable'] = data.loc[i, 'abc']
    data.loc[i, 'abc'] = 0

    data.loc[12 + i, 'SD'] = data.loc[12 + i, 'Mean']
    data.loc[12 + i, 'Mean'] = data.loc[12 + i, 'Variable']
    data.loc[12 + i, 'Variable'] = data.loc[12 + i, 'abc']
    data.loc[12 + i, 'abc'] = 1

now we merge the table with the given label

In [60]:
df = pd.read_excel('../02. Datasets/variable__label_correspondence.xlsx')
df['varlab'] = df['varlab'].str.replace(r'^s\d+q\d+_\d+\s+', '', regex=True)
df['varlab'] = df['varlab'].str.replace(r's22q1_15a ', '') # s22q1_15a 
df['varlab'] = df['varlab'].str.replace(r'id11 ', '')

In [61]:
data['Variable'].replace('edchild pe~t', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)
data['Variable'].replace('eth hausa	', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)


In [62]:
data['Variable'] = data['Variable'] \
    .str.replace(' ', '_') \
    .str.replace('edchild_pe~t', 'edchild_percent') \
    .str.replace('eth_hausa', 'eth_hausa') \
    .str.replace('hhmem_no', 'hhmem_no') \
    .str.replace('accesscell~e', 'accesscellphone')


In [63]:
data = data.merge(df, left_on='Variable', right_on='name', how='left')

In [64]:
dataframe = data[['Variable', 'varlab', 'Mean', 'SD', 'abc']]

In [65]:
df_abc_0 = dataframe.iloc[:12].reset_index(drop=True)
df_abc_1 = dataframe[12:][['varlab', 'Mean', 'SD']].reset_index(drop=True)

df_abc_1.rename(columns={'Mean': 'Mean with abc', 'SD': 'SD with abc'}, inplace=True)

result_df = pd.merge(df_abc_0, df_abc_1, on='varlab', how='left')
result_df.rename(columns={'Mean': 'Mean without abc', 'SD': 'SD without abc'}, inplace=True)

In [66]:
result_df.drop(columns = ['abc'], inplace = True)

In [67]:
result_df

,Variable,varlab,Mean without abc,SD without abc,Mean with abc,SD with abc
0,age,age,37.86,13.10,37.18,11.76
1,hhhead,Are you the household head?,0.560,0.497,0.547,0.498
2,eth_hausa,Respondent is Hausa,0.715,0.452,0.721,0.449
3,hhmem_no,Number of household members,8.422,4.054,8.328,4.074
4,edchild_percent,Percentage of children under 15 who have some ...,0.279,0.276,0.269,0.270
5,assets,Number of asset categories owned by household,4.990,1.609,4.979,1.575
6,drought,Household experienced drought in past year,0.385,0.487,0.380,0.486
7,cellphone,Household owns a cell phone (excluding group p...,0.296,0.457,0.295,0.457
8,accesscellphone,Access to household or village-level cell phone,0.763,0.426,0.798,0.402
9,usecellphone,Respondent has used cell phone since last harvest,0.542,0.499,0.573,0.495


In [68]:
difference = pd.read_stata('../02. Datasets/Table1_PanelA_dta.dta')
difference.columns = difference.iloc[1]
# Drop the first row
difference = difference.drop(0).reset_index(drop=True)
difference = difference.drop(0).reset_index(drop=True)

result_data = pd.DataFrame(difference.iloc[1])
result_data['std'] = difference.iloc[2]
result_data.columns = result_data.iloc[0]

result_data.drop('VARIABLES', inplace=True)

result_data.reset_index(drop = False, inplace = True)
result_data.reset_index(drop = True, inplace = True)

In [69]:
result_data.rename(columns = {1: 'Name', 'abc': 'Diff', '': 'std'}, inplace = True)

In [70]:
result_data.reset_index(drop = True, inplace = True)

In [71]:
result_df = pd.merge(result_df, result_data, left_on= 'Variable', right_on='Name', how='left')

In [72]:
result_df.drop(columns = ['Variable', 'Name'], inplace = True)
result_df.rename(columns = {'varlab': 'Variable'}, inplace = True)

In [73]:
latex_table  = result_df.to_latex(index=False, escape=False)

In [74]:
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
age & 37.86 & 13.10 & 37.18 & 11.76 & -0.36 & (0.93) \\
Are you the household head? & 0.560 & 0.497 & 0.547 & 0.498 & -0.01 & (0.02) \\
Respondent is Hausa & 0.715 & 0.452 & 0.721 & 0.449 & 0.01 & (0.03) \\
Number of household members & 8.422 & 4.054 & 8.328 & 4.074 & 0.02 & (0.25) \\
Percentage of children under 15 who have some education & 0.279 & 0.276 & 0.269 & 0.270 & -0.00 & (0.02) \\
Number of asset categories owned by household & 4.990 & 1.609 & 4.979 & 1.575 & -0.03 & (0.10) \\
Household experienced drought in past year & 0.385 & 0.487 & 0.380 & 0.486 & -0.03 & (0.03) \\
Household owns a cell phone (excluding group phone) & 0.296 & 0.457 & 0.295 & 0.457 & -0.00 & (0.03) \\
Access to household or village-level cell phone & 0.763 & 0.426 & 0.798 & 0.402 & 0.04* & (0.02) \\
Respondent has used cell phone since last harvest & 0.542 & 0.499 & 0.573 &

In [75]:
parts = latex_table.split("\\midrule", 1)
new_line = "\\multicolumn{7}{l}{\\textbf{Panel A: pre-program household characteristics}}  \\\\"
modified_table = parts[0] + "\\midrule\n" + new_line + parts[1]


In [76]:
print(modified_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
\multicolumn{7}{l}{\textbf{Panel A: pre-program household characteristics}}  \\
age & 37.86 & 13.10 & 37.18 & 11.76 & -0.36 & (0.93) \\
Are you the household head? & 0.560 & 0.497 & 0.547 & 0.498 & -0.01 & (0.02) \\
Respondent is Hausa & 0.715 & 0.452 & 0.721 & 0.449 & 0.01 & (0.03) \\
Number of household members & 8.422 & 4.054 & 8.328 & 4.074 & 0.02 & (0.25) \\
Percentage of children under 15 who have some education & 0.279 & 0.276 & 0.269 & 0.270 & -0.00 & (0.02) \\
Number of asset categories owned by household & 4.990 & 1.609 & 4.979 & 1.575 & -0.03 & (0.10) \\
Household experienced drought in past year & 0.385 & 0.487 & 0.380 & 0.486 & -0.03 & (0.03) \\
Household owns a cell phone (excluding group phone) & 0.296 & 0.457 & 0.295 & 0.457 & -0.00 & (0.03) \\
Access to household or village-level cell phone & 0.763 & 0.426 & 0.798 & 0.402 & 0.04* & (0.02

In [100]:
data = pd.read_stata('../02. Datasets/Table1_PanelB_mean_std.dta')
data.columns = data.iloc[0]
# Drop the first row
data = data.drop(0).reset_index(drop=True)
j = 4
for i in range(1, j):
    data.loc[i, 'SD'] = data.loc[i, 'Mean']
    data.loc[i, 'Mean'] = data.loc[i, 'Variable']
    data.loc[i, 'Variable'] = data.loc[i, 'abc']
    data.loc[i, 'abc'] = 0

    data.loc[j + i, 'SD'] = data.loc[j + i, 'Mean']
    data.loc[j + i, 'Mean'] = data.loc[j + i, 'Variable']
    data.loc[j + i, 'Variable'] = data.loc[j + i, 'abc']
    data.loc[j + i, 'abc'] = 1

df = pd.read_excel('../02. Datasets/variable__label_correspondence_teacher.xlsx')
data['Variable'].replace('femaleteac~r', 'femaleteacher', inplace = True)


data = data.merge(df, left_on='Variable', right_on='name', how='left')
dataframe = data[['Variable', 'varlab', 'Mean', 'SD', 'abc']]
# print(dataframe)
df_abc_0 = dataframe.iloc[:j].reset_index(drop=True)
df_abc_1 = dataframe.iloc[j:][['varlab', 'Mean', 'SD']].reset_index(drop=True)

df_abc_1.rename(columns={'Mean': 'Mean with abc', 'SD': 'SD with abc'}, inplace=True)
# print(df_abc_0)
result_df = pd.merge(df_abc_0, df_abc_1, on='varlab', how='left')
result_df.rename(columns={'Mean': 'Mean without abc', 'SD': 'SD without abc'}, inplace=True)
result_df.drop(columns = ['abc'], inplace = True)
result_df

,Variable,varlab,Mean without abc,SD without abc,Mean with abc,SD with abc
0,levelno,Level of Instruction of Teacher,8.323,2.084,8.572,1.779
1,teacherage,Age of Teacher,33.06,9.158,32.71,8.067
2,femaleteacher,Female Teacher,0.317,0.467,0.368,0.484
3,local,Teacher from Same Village,0.757,0.430,0.682,0.467


In [101]:

difference = pd.read_stata('../02. Datasets/Table1_PanelB_dta.dta')
difference.columns = difference.iloc[1]
# Drop the first row
difference = difference.drop(0).reset_index(drop=True)
difference = difference.drop(0).reset_index(drop=True)

result_data = pd.DataFrame(difference.iloc[1])
result_data['std'] = difference.iloc[2]
result_data.columns = result_data.iloc[0]

result_data.drop('VARIABLES', inplace=True)

result_data.reset_index(drop = False, inplace = True)
result_data.reset_index(drop = True, inplace = True)
result_data.rename(columns = {1: 'Name', 'abc': 'Diff', '': 'std'}, inplace = True)
result_data.reset_index(drop = True, inplace = True)
result_data

VARIABLES,Name,Diff,std
0,levelno,0.08,(0.22)
1,teacherage,-0.31,(1.18)
2,femaleteacher,0.06,(0.04)
3,local,-0.02,(0.05)


In [102]:
result_df = pd.merge(result_df, result_data, left_on= 'Variable', right_on='Name', how='left')
result_df.drop(columns = ['Variable', 'Name'], inplace = True)
result_df.rename(columns = {'varlab': 'Variable'}, inplace = True)
latex_table  = result_df.to_latex(index=False, escape=False)

In [103]:
parts = latex_table.split("\\midrule", 1)
parts = parts[1].split("\\bottomrule", 1)
new_line = "\\multicolumn{7}{l}{\\textbf{Panel B: pre-program Teacher characteristics}}  \\\\"
added_table1 = new_line + parts[0]


In [104]:
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
Level of Instruction of Teacher & 8.323 & 2.084 & 8.572 & 1.779 & 0.08 & (0.22) \\
Age of Teacher & 33.06 & 9.158 & 32.71 & 8.067 & -0.31 & (1.18) \\
Female Teacher & 0.317 & 0.467 & 0.368 & 0.484 & 0.06 & (0.04) \\
Teacher from Same Village & 0.757 & 0.430 & 0.682 & 0.467 & -0.02 & (0.05) \\
\bottomrule
\end{tabular}



In [105]:
data = pd.read_stata('../02. Datasets/Table1_PanelC_mean_std.dta')
data.columns = data.iloc[0]
# Drop the first row
data = data.drop(0).reset_index(drop=True)
j = 2
for i in range(1, j):
    data.loc[i, 'SD'] = data.loc[i, 'Mean']
    data.loc[i, 'Mean'] = data.loc[i, 'Variable']
    data.loc[i, 'Variable'] = data.loc[i, 'abc']
    data.loc[i, 'abc'] = 0

    data.loc[j + i, 'SD'] = data.loc[j + i, 'Mean']
    data.loc[j + i, 'Mean'] = data.loc[j + i, 'Variable']
    data.loc[j + i, 'Variable'] = data.loc[j + i, 'abc']
    data.loc[j + i, 'abc'] = 1

df = pd.read_excel('../02. Datasets/variable__label_correspondence_test_score.xlsx')

data = data.merge(df, left_on='Variable', right_on='name', how='left')
dataframe = data[['Variable', 'varlab', 'Mean', 'SD', 'abc']]

df_abc_0 = dataframe.iloc[:j].reset_index(drop=True)
df_abc_1 = dataframe.iloc[j:][['varlab', 'Mean', 'SD']].reset_index(drop=True)

df_abc_1.rename(columns={'Mean': 'Mean with abc', 'SD': 'SD with abc'}, inplace=True)
# print(df_abc_0)
result_df = pd.merge(df_abc_0, df_abc_1, on='varlab', how='left')
result_df.rename(columns={'Mean': 'Mean without abc', 'SD': 'SD without abc'}, inplace=True)
result_df.drop(columns = ['abc'], inplace = True)
result_df

,Variable,varlab,Mean without abc,SD without abc,Mean with abc,SD with abc
0,writez1,Baseline literacy test Z-score,-1.03e-08,1.000,-0.0269,0.886
1,mathz1,Baseline numeracy test Z-score,-6.69e-09,1.000,-0.0712,0.816


In [106]:

difference = pd.read_stata('../02. Datasets/Table1_PanelC_dta.dta')
difference.columns = difference.iloc[1]
# Drop the first row
difference = difference.drop(0).reset_index(drop=True)
difference = difference.drop(0).reset_index(drop=True)

result_data = pd.DataFrame(difference.iloc[1])
result_data['std'] = difference.iloc[2]
result_data.columns = result_data.iloc[0]

result_data.drop('VARIABLES', inplace=True)

result_data.reset_index(drop = False, inplace = True)
result_data.reset_index(drop = True, inplace = True)
result_data.rename(columns = {1: 'Name', 'abc': 'Diff', '': 'std'}, inplace = True)
result_data.reset_index(drop = True, inplace = True)
result_data

VARIABLES,Name,Diff,std
0,writez1,-0.02,(0.04)
1,mathz1,-0.06,(0.05)


In [107]:
result_df = pd.merge(result_df, result_data, left_on= 'Variable', right_on='Name', how='left')
result_df.drop(columns = ['Variable', 'Name'], inplace = True)
result_df.rename(columns = {'varlab': 'Variable'}, inplace = True)
latex_table  = result_df.to_latex(index=False, escape=False)

In [108]:
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
Baseline literacy test Z-score & -1.03e-08 & 1.000 & -0.0269 & 0.886 & -0.02 & (0.04) \\
Baseline numeracy test Z-score & -6.69e-09 & 1.000 & -0.0712 & 0.816 & -0.06 & (0.05) \\
\bottomrule
\end{tabular}



In [109]:
parts = latex_table.split("\\midrule", 1)
parts = parts[1].split("\\bottomrule", 1)
new_line = "\\multicolumn{7}{l}{\\textbf{Panel C: pre-program Test-Score characteristics}}  \\\\"
added_table2 = new_line + parts[0]


In [110]:
new_table = modified_table.split('\\bottomrule', 1)
new_table = new_table[0] + added_table1 + added_table2 + "\\bottomrule" + new_table[1]
print(new_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
\multicolumn{7}{l}{\textbf{Panel A: pre-program household characteristics}}  \\
age & 37.86 & 13.10 & 37.18 & 11.76 & -0.36 & (0.93) \\
Are you the household head? & 0.560 & 0.497 & 0.547 & 0.498 & -0.01 & (0.02) \\
Respondent is Hausa & 0.715 & 0.452 & 0.721 & 0.449 & 0.01 & (0.03) \\
Number of household members & 8.422 & 4.054 & 8.328 & 4.074 & 0.02 & (0.25) \\
Percentage of children under 15 who have some education & 0.279 & 0.276 & 0.269 & 0.270 & -0.00 & (0.02) \\
Number of asset categories owned by household & 4.990 & 1.609 & 4.979 & 1.575 & -0.03 & (0.10) \\
Household experienced drought in past year & 0.385 & 0.487 & 0.380 & 0.486 & -0.03 & (0.03) \\
Household owns a cell phone (excluding group phone) & 0.296 & 0.457 & 0.295 & 0.457 & -0.00 & (0.03) \\
Access to household or village-level cell phone & 0.763 & 0.426 & 0.798 & 0.402 & 0.04* & (0.02

In [111]:
# File path
file_path = '../manuscript/Tables/contamination_of_the_randomization.tex'

# Save the LaTeX table to a file
with open(file_path, 'w') as f:
    f.write(new_table)

file_path

'../manuscript/Tables/contamination_of_the_randomization.tex'